<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SPD-and-HFM" data-toc-modified-id="SPD-and-HFM-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SPD and HFM</a></span></li></ul></div>

In [ ]:
# ======================================================================
# 1. IMPORTS
# ======================================================================
import os
import copy
import shutil
import sys
import glob
import time
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Geostatistics, HFM, and other packages
import pygeostat as gs
from agd import Eikonal
from agd.Metrics import Riemann

from tqdm import tqdm
from scipy import spatial
from scipy.signal import remez, minimum_phase, freqz, group_delay
from scipy import io
from decimal import Decimal
from statsmodels.graphics import tsaplots
from sklearn.metrics import r2_score

# Neural network packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# For hyperparameter tuning
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from sklearn.model_selection import train_test_split

## SPD and HFM

In [ ]:
# ======================================================================
# 2. DIRECTORIES AND DATA LOADING
# ======================================================================
outdir = 'Output/'
exedir = 'exe/'

# Load the grid definition from a file (adjust filename as needed)
griddef = gs.GridDef(grid_file='griddef.txt')
print(griddef)

# Load borehole (or sample) data (adjust filename/columns as needed)
bhdata = gs.DataFile('cudata.dat', griddef=griddef, readfl=True)


In [ ]:
LVA =gs.DataFile('./Output/LVA.out', griddef = griddef)
LVA.addcoord()
LVA.data.describe()

In [ ]:
LVA.data = LVA.data.rename(columns={'angle1': 'strike','angle2': 'dip','angle3': 'plunge', 
                                    'ranges12': 'r1',  'ranges13' :'r2'} )


In [ ]:
gs.write_gslib(LVA.data, './Output/LVA2.out')
LVA =gs.DataFile('./Output/LVA2.out', griddef = griddef)
LVA.data.describe()

In [ ]:
# For example, ensure they’re not below 0.001
minval = 1e-3
LVA.data['r1'] = LVA.data['r1'].clip(lower=minval)
LVA.data['r2'] = LVA.data['r2'].clip(lower=minval)


In [ ]:
LVA.data.describe()

In [ ]:
def generate_grid_points_rev3D(griddef, order='GSLIB'):
    """
    Generate (y, z, x) coords for the FULL grid, 
    shaped (nz, ny, nx) if you want that ordering.
    
    We'll interpret:
      axis0 => y,
      axis1 => z,
      axis2 => x,
    consistent with dims=[nz,ny,nx] in HFM.
    """
    ny, nz, nx = griddef.ny, griddef.nz, griddef.nx
    
    # y-coords
    y_vals = np.linspace(griddef.ylimits[0],
                         griddef.ylimits[1] - griddef.ysiz,
                         ny)
    # z-coords
    z_vals = np.linspace(griddef.zlimits[0],
                         griddef.zlimits[1] - griddef.zsiz,
                         nz)
    # x-coords
    x_vals = np.linspace(griddef.xlimits[0],
                         griddef.xlimits[1] - griddef.xsiz,
                         nx)
    
    # indexing='ij' => G shape (ny, nz, nx) if we do Y, Z, X
    # but we actually want shape (nz, ny, nx), so let's reorder:
    Y, Z, X = np.meshgrid(y_vals, z_vals, x_vals, indexing='ij')
    # Y.shape => (ny, nz, nx)
    # We'll interpret axis0 => y, axis1 => z, axis2 => x
    
    if order=='GSLIB':
        coords = np.column_stack([Y.ravel(order='C'),
                                  Z.ravel(order='C'),
                                  X.ravel(order='C')])
    else:
        coords = np.column_stack([Y.ravel(order='F'),
                                  Z.ravel(order='F'),
                                  X.ravel(order='F')])
    return coords


def landmark_points_rev3D(Ly, Lz, Lx, griddef, plot=False):
    """
    3D extension for landmark points, specifically in (y,z,x) order.
    
    Ly, Lz, Lx = number of landmark seeds along y,z,x.
    We'll interpret dims=[nz, ny, nx].
    """
    ny, nz, nx = griddef.ny, griddef.nz, griddef.nx
    
    # intervals
    interval_y = ny/(Ly + 1)
    interval_z = nz/(Lz + 1)
    interval_x = nx/(Lx + 1)
    
    # actual domain range
    y_vals = np.linspace(griddef.ylimits[0] + interval_y*griddef.ysiz,
                         griddef.ylimits[1] - interval_y*griddef.ysiz,
                         Ly)
    z_vals = np.linspace(griddef.zlimits[0] + interval_z*griddef.zsiz,
                         griddef.zlimits[1] - interval_z*griddef.zsiz,
                         Lz)
    x_vals = np.linspace(griddef.xlimits[0] + interval_x*griddef.xsiz,
                         griddef.xlimits[1] - interval_x*griddef.xsiz,
                         Lx)
    
    # mesh in (y,z,x) => shape (Ly, Lz, Lx)
    Yv, Zv, Xv = np.meshgrid(y_vals, z_vals, x_vals, indexing='ij')
    
    if plot:
        import matplotlib.pyplot as plt
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(Yv, Zv, Xv, c='k')
        ax.set_title(f"Landmark Points: (Ly={Ly}, Lz={Lz}, Lx={Lx})")
        plt.show()
    
    return Yv, Zv, Xv


In [ ]:
def angles_to_unit_vector_3D(strike_deg, dip_deg, plunge_deg):
    """
    Convert (strike, dip, plunge) in degrees -> (uX, uY, uZ).
    But be mindful we interpret the final 3D as (y,z,x)...

    Suppose you keep the same formula:
      ux = sin(strike)*cos(dip)
      uy = cos(strike)*cos(dip)
      uz = -sin(dip)
    
    We'll rename them as uX, uY, uZ for clarity, but watch your usage.
    """
    s = np.radians(strike_deg)
    d = np.radians(dip_deg)
    # ignoring plunge for now
    
    uX = np.sin(s)*np.cos(d)
    uY = np.cos(s)*np.cos(d)
    uZ = -np.sin(d)
    
    length = np.sqrt(uX**2 + uY**2 + uZ**2)
    if length>1e-12:
        uX, uY, uZ = uX/length, uY/length, uZ/length
    return uX, uY, uZ


In [ ]:
def SPD_HFM_3D_rev10(hfmIn, LVA, griddef, N_data,
                     order='GSLIB', geodesics=False, tips=(4,4,4),
                     L_points=(10,10,10)):
    """
    3D version of your SPD_HFM_rev10, using (y,z,x) axis order.
    
    We interpret:
      dims=[nz, ny, nx] in hfmIn.SetRect
      arrays => shape (nz, ny, nx, order='C')
    """
    import numpy as np
    from tqdm import tqdm
    
    ny, nz, nx = griddef.ny, griddef.nz, griddef.nx
    
    # 1) Extract columns
    if 'r1' not in LVA.data.columns or 'r2' not in LVA.data.columns:
        raise ValueError("LVA must have r1, r2 columns!")
    if 'strike' not in LVA.data.columns or 'dip' not in LVA.data.columns:
        raise ValueError("Need strike, dip columns!")
    
    # Clipping to avoid zero
    LVA.data['r1'] = LVA.data['r1'].clip(lower=1e-3)
    LVA.data['r2'] = LVA.data['r2'].clip(lower=1e-3)
    
    strike_arr = LVA['strike'].values
    dip_arr    = LVA['dip'].values
    r1_arr     = LVA['r1'].values
    r2_arr     = LVA['r2'].values
    
    if len(strike_arr)!=N_data:
        raise ValueError("Mismatch in LVA size vs N_data!")
    
    # 2) Reshape to (nz, ny, nx) in row-major => 'C'
    # But note typical usage => the first index is z, second is y, third is x
    # => shape (nz, ny, nx)
    strike_3d = strike_arr.reshape(nz, ny, nx, order='C')
    dip_3d    = dip_arr.reshape(nz, ny, nx, order='C')
    r1_3d     = r1_arr.reshape(nz, ny, nx, order='C')
    r2_3d     = r2_arr.reshape(nz, ny, nx, order='C')
    
    # Build orientation arrays
    uX_3d = np.zeros((nz, ny, nx), dtype=float)
    uY_3d = np.zeros((nz, ny, nx), dtype=float)
    uZ_3d = np.zeros((nz, ny, nx), dtype=float)
    
    for k in range(nz):
        for j in range(ny):
            for i in range(nx):
                st_deg = strike_3d[k,j,i]
                dp_deg = dip_3d[k,j,i]
                ux, uy, uz = angles_to_unit_vector_3D(st_deg, dp_deg, 0.0)
                uX_3d[k,j,i] = ux
                uY_3d[k,j,i] = uy
                uZ_3d[k,j,i] = uz
    
    alpha = 1.0 / r1_3d
    beta  = 1.0 / r2_3d
    
    # 3) Build metric => Riemann.needle((uX_3d,uY_3d,uZ_3d), alpha,beta).dual()
    hfmIn['metric'] = Riemann.needle(
        (uX_3d, uY_3d, uZ_3d),
        alpha,
        beta
    ).dual()
    
    if geodesics:
        hfmIn.SetUniformTips(tips)
    hfmIn['exportValues'] = 1
    hfmIn['exportGeodesicFlow'] = 0
    
    path_vis = []
    
    # 4) SPD: Full vs. Landmark approach
    if L_points==0:
        # Full NxNyNz => NxN SPD
        coords_3d = generate_grid_points_rev3D(griddef, order=order)  # shape => (N_data,3)
        # coords_3d[:,0] => y, coords_3d[:,1]=> z, coords_3d[:,2]=> x
        N_h_spd = N_data
        # NxN SPD => watch memory if large
        h_spd_mtx = np.zeros((N_data, N_data), dtype=np.float32)
        
        # partial vector
        N_pairs = np.sum(range(N_data))
        h_spd_vecc = np.zeros((N_pairs,1), dtype=np.float32)
        i = 0
        for i1 in tqdm(range(N_h_spd-1)):
            sy, sz, sx = coords_3d[i1,:]
            hfmIn['seed'] = [sy, sz, sx]  # (y,z,x)
            hfmOut = hfmIn.Run()
            
            if geodesics:
                path_vis.append(hfmOut['geodesics'])
            
            # flatten
            if order=='GSLIB':
                dist_flat = hfmOut['values'].flatten(order='C')
            else:
                dist_flat = hfmOut['values'].flatten(order='F')
            
            i2_len = N_data - (i1+1)
            h_spd_vecc[i:i+i2_len,0] = dist_flat[i1+1:]
            h_spd_mtx[i1+1:, i1] = h_spd_vecc[i:i+i2_len,0]
            i += i2_len
        
        # Mirror
        h_spd_mtx = h_spd_mtx + h_spd_mtx.T
    
    else:
        # Landmark approach => shape [N_landmarks, N_data]
        Ly, Lz, Lx = L_points
        Yv, Zv, Xv = landmark_points_rev3D(Ly, Lz, Lx, griddef, plot=False)
        
        if order=='GSLIB':
            source_y = Yv.flatten(order='C')
            source_z = Zv.flatten(order='C')
            source_x = Xv.flatten(order='C')
        else:
            source_y = Yv.flatten(order='F')
            source_z = Zv.flatten(order='F')
            source_x = Xv.flatten(order='F')
        
        N_h_spd = Ly*Lz*Lx
        h_spd_mtx = np.zeros((N_h_spd, N_data), dtype=np.float32)
        
        for i1 in tqdm(range(N_h_spd)):
            sy, sz, sx = source_y[i1], source_z[i1], source_x[i1]
            hfmIn['seed'] = [sy, sz, sx]
            hfmOut = hfmIn.Run()
            
            if geodesics:
                path_vis.append(hfmOut['geodesics'])
            
            if order=='GSLIB':
                dist_flat = hfmOut['values'].flatten(order='C')
            else:
                dist_flat = hfmOut['values'].flatten(order='F')
            
            h_spd_mtx[i1,:] = dist_flat
    
    return h_spd_mtx, path_vis


In [ ]:
if __name__ == "__main__":
    import pygeostat as gs

    hfmIn = Eikonal.dictIn({
        'arrayOrdering': 'RowMajor',  # similar to 2D
        'model': 'Riemann3',         # now 3D
        'seedValue': 0,
        'geodesicStep':0.1,
        'order': 5,
    })

    # Example: sides=( (ymin,ymax),(zmin,zmax),(xmin,xmax) ), dims=[nz, ny, nx]
    hfmIn.SetRect(
        sides=[[griddef.ylimits[0], griddef.ylimits[1]],
               [griddef.zlimits[0], griddef.zlimits[1]],
               [griddef.xlimits[0], griddef.xlimits[1]]],
        dims=[griddef.nz, griddef.ny, griddef.nx]
    )
    
    # N_data = griddef.count()
    # SPD call
    h_spd_mtx, path_vis = SPD_HFM_3D_rev10(
        hfmIn, LVA, griddef,
        N_data=griddef.count(),
        order='GSLIB',
        geodesics=True,
        tips=(4,4,4),
        L_points=(5,5,5)  
    )
    print("SPD shape:", h_spd_mtx.shape)
    if path_vis:
        print("Geodesics found:", len(path_vis))
    



In [ ]:
# df_spd = pd.DataFrame(h_spd_mtx.T)
# gs.write_gslib(df_spd, 'spd3d.out')   
# data  = gs.DataFile('spd3d.out')

In [ ]:
# import matplotlib.pyplot as plt

# # Select 9 variables from 0 to 120 in increments of 15
# variables = [str(i) for i in range(0, 8, 1)]  # ['0', '15', '30', '45', '60', '75', '90', '105', '120']

# # Define the slice index (e.g., z=10)
# slice_number = 15

# # Create a 3x3 grid of subplots
# fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(12, 12))

# # Flatten the 2D array of axes to iterate easily
# axes_flat = axes.ravel()

# # Loop through each variable and axis
# for var, ax in zip(variables, axes_flat):
#     gs.slice_plot(
#         data,
#         var=var,
#         griddef=griddef,
#         ax=ax,
#         orient='xy',
#         slice_number=slice_number,
#         cbar=False,
#         title=f"Distance from seed[0]\nz={slice_number}, Var={var}"
#     )

# plt.tight_layout()
# plt.show()


In [ ]:
x,y,z = griddef.get_coordinates()
grid_coords= np.hstack((x.reshape(len(x),1),y.reshape((len(y)),1),z.reshape((len(z)),1)))

In [ ]:
import numpy as np
import pandas as pd
from scipy import spatial
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Hyperopt
from hyperopt import fmin, tpe, hp, Trials

# ==================================================================================
# 1. SPD Computation + Standardization
# ==================================================================================
mean_value = np.mean(h_spd_mtx)
std_value = np.std(h_spd_mtx)

# Scale SPD matrix
h_spd_HFM_scaled = (h_spd_mtx - mean_value) / (std_value + 1e-12)

# ----------------------------------------------------------------------------------
# Create a DataFrame for grid coordinates
# ----------------------------------------------------------------------------------
df1 = pd.DataFrame(grid_coords, columns=['X', 'Y', 'Z'])

# Optionally normalize coordinates if you want them as features
lon = df1['X'].values
lat = df1['Y'].values
elev = df1['Z'].values

normalized_lon = (lon - lon.min()) / (lon.max() - lon.min())
normalized_lat = (lat - lat.min()) / (lat.max() - lat.min())
normalized_elev = (elev - elev.min()) / (elev.max() - elev.min())

s_test = np.vstack((normalized_lon, normalized_lat, normalized_elev)).T

# ==================================================================================
# Outer 5-Fold Loop (Nested CV: Outer Loop)
# ==================================================================================
for fold in range(1, 6):
    print(f"\n========================")
    print(f" OUTER FOLD {fold}/5")
    print(f"========================\n")

    # ------------------------------------------------------------------------------
    # 2. Load outer-train data
    # ------------------------------------------------------------------------------
    Traindat = gs.DataFile(f'Training0{fold}.dat', x='X', y='Y', z='Z')
    df_train = Traindat.data

    # Map training points to SPD matrix rows
    near_points = df1[['X', 'Y', 'Z']].values
    tree = spatial.KDTree(near_points)
    idx = tree.query(df_train[['X', 'Y', 'Z']].values)[1]

    # Group by neighbor index and compute mean of 'Cu'
    df_train_neighbor = df_train.assign(neighbor=idx)
    df_cu = df_train_neighbor.groupby('neighbor')['Cu'].mean()

    idx_new = df_cu.index.values
    pm25 = df_cu.values  # Target variable

    # Build feature matrix
    phi_combined = h_spd_HFM_scaled.T  # shape => (N_data, #landmarks)
    X_full = phi_combined[idx_new, :]
    y_full = pm25.reshape(-1, 1)

    # ------------------------------------------------------------------------------
    # 3. Define model-creation function
    # ------------------------------------------------------------------------------
    def create_model(hparams, input_dim):
        """
        Build a Keras model using hyperparameters.
        Three hidden layers, each with Dropout and l1_l2 regularization.
        """
        model = Sequential()

        # Hidden layer 1
        model.add(
            Dense(
                int(hparams['units']),
                input_dim=input_dim,
                kernel_initializer='he_uniform',
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-6, l2=1e-4)
            )
        )
        model.add(Dropout(hparams['dropout']))

        # Hidden layer 2
        model.add(
            Dense(
                int(hparams['units']),
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-6, l2=1e-4)
            )
        )
        model.add(Dropout(hparams['dropout']))

        # Hidden layer 3
        model.add(
            Dense(
                int(hparams['units']),
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-6, l2=1e-4)
            )
        )
        model.add(Dropout(hparams['dropout']))

        # Output layer
        model.add(Dense(1, activation='linear'))

        optimizer = Adam(learning_rate=hparams['learning_rate'])
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        return model

    # ------------------------------------------------------------------------------
    # 4. Inner 5-Fold CV for hyperparameter tuning (via Hyperopt)
    # ------------------------------------------------------------------------------
    def hyperopt_objective(hparams):
        """
        For each set of hyperparams, perform 5-fold CV on (X_full, y_full).
        Return the average MSE across the 5 folds as the objective to minimize.
        """
        epochs = int(hparams['epochs'])
        batch_size = int(hparams['batch_size'])

        # Change the random seed here for each outer fold
        kf = KFold(n_splits=5, shuffle=True, random_state=42 + fold)
        fold_mses = []

        for train_index, val_index in kf.split(X_full):
            X_tr, X_val = X_full[train_index], X_full[val_index]
            y_tr, y_val = y_full[train_index], y_full[val_index]

            model = create_model(hparams, input_dim=X_full.shape[1])
            model.fit(X_tr, y_tr, epochs=epochs, batch_size=batch_size, verbose=0)

            y_val_pred = model.predict(X_val)
            fold_mses.append(mean_squared_error(y_val, y_val_pred))

        return np.mean(fold_mses)

    # Define Hyperparameter search space
    search_space = {
        'units': hp.quniform('units', 400, 1000, 200),
        'dropout': hp.uniform('dropout', 0.2, 0.5),
        'learning_rate': hp.loguniform('learning_rate', np.log(1e-5), np.log(1e-2)),
        'epochs': hp.quniform('epochs', 200, 800, 200),
        'batch_size': hp.quniform('batch_size', 16, 64, 16),
    }

    # Run hyperparameter optimization with a different seed each outer fold
    trials = Trials()
    best = fmin(
        fn=hyperopt_objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=5,
        trials=trials,
        # Use a fold-dependent seed, e.g., 42+fold
        rstate=np.random.default_rng(42 + fold)
    )

    # Convert selected params to correct data types
    best['units'] = int(best['units'])
    best['epochs'] = int(best['epochs'])
    best['batch_size'] = int(best['batch_size'])

    print("Best hyperparameters from inner CV:")
    print(best)

    # ------------------------------------------------------------------------------
    # 5. Retrain final model on the FULL outer-train data
    # ------------------------------------------------------------------------------
    def create_regularized_model_final(best_params, input_dim):
        """
        Build a final model using the best hyperparams
        from the inner CV, with l1/l2 fixed.
        """
        model = Sequential()

        # 3 hidden layers
        for _ in range(3):
            model.add(Dense(
                best_params['units'],
                input_dim=input_dim,
                kernel_initializer='he_uniform',
                activation='relu',
                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=1e-6, l2=1e-4)
            ))
            model.add(Dropout(best_params['dropout']))
            # Only specify input_dim on the first layer
            input_dim = None

        # Output layer
        model.add(Dense(1, activation='linear'))

        optimizer = Adam(learning_rate=best_params['learning_rate'])
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        return model

    # Build and train the final model on the entire outer-train data
    final_model = create_regularized_model_final(best, input_dim=X_full.shape[1])
    final_model.fit(
        X_full,
        y_full,
        epochs=best['epochs'],
        batch_size=best['batch_size'],
        verbose=0
    )

    # ------------------------------------------------------------------------------
    # 6. Predict on the entire domain
    # ------------------------------------------------------------------------------
    X_RBF_pred = phi_combined  # entire domain
    PM25_pred = final_model.predict(X_RBF_pred)

    # Clip negative predictions (optional)
    PM25_pred[PM25_pred < 0] = 0

    outfile = f'DeepKriging_{fold}.out'
    gs.write_gslib(pd.DataFrame(PM25_pred), outfile)
    print(f"Domain predictions saved: {outfile}")

    # ------------------------------------------------------------------------------
    # 7. Evaluate on validation set
    # ------------------------------------------------------------------------------
    testfile = gs.DataFile(flname=f'Validation0{fold}.dat', griddef=griddef, readfl=True)
    DeepKriging = gs.DataFile(f'DeepKriging_{fold}.out', griddef=griddef)

    idx, ingrid = griddef.get_index(
        x=testfile.data['X'],
        y=testfile.data['Y'],
        z=testfile.data['Z']
    )

    testfile.data = testfile.data[idx >= 0]
    idx = idx[idx >= 0]

    # Print R^2 score on the test set
    print(
        r2_score(
            testfile['Cu'],
            DeepKriging.data['0'][idx].values
        )
    )


In [ ]:
# ======================================================================
# 9. COND. DEEP KRIGING 
# ======================================================================
for fold in range(1,6):
    deepkriging = gs.DataFile(f'DeepKriging_{fold}.out', griddef=griddef)
    moment = gs.DataFile(f'./IDW/Debug_{fold}.out', x='APX', y='APY')

    # Compute scaled variance for weighting
    moment.data['Variance'] = moment.data['Local std.dev.']**2
    moment.data['Scaled_Variance'] = (
        moment.data['Variance'] - moment.data['Variance'].min()
    ) / (
        moment.data['Variance'].max() - moment.data['Variance'].min()
    )

    weight = moment['Scaled_Variance'] ** 0.25
    cond_DK = deepkriging['0'] * np.array(weight) + moment['Local Mean'] * (1 - np.array(weight))

    # Save conditional DK
    gs.write_gslib(pd.DataFrame(cond_DK), f'./IDW/Weighted_DK_{fold}.out')


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd

OK_R2 = 0
DeepKriging_R2 = 0
Cond_DK_R2 = 0
OK_Ensemble = 0
DeepKriging_Ensemble = 0
Cond_DK_Ensemble = 0

for fold in range(1, 6):
    # Read the validation file
    testfile = gs.DataFile(flname='Validation0{}.dat'.format(fold),
                           griddef=griddef, readfl=True)

    # Read the kriging outputs
    OK = gs.DataFile('OK_{}.out'.format(fold), griddef=griddef)
    # Ensure no negative estimates
    OK.data.loc[OK.data['Estimate'] < 0, 'Estimate'] = 0

    DeepKriging = gs.DataFile('DeepKriging_{}.out'.format(fold), griddef=griddef)
    Cond_DK = gs.DataFile('./IDW/Weighted_DK_{}.out'.format(fold), griddef=griddef)

    # Get indices for testfile data points
    idx, ingrid = griddef.get_index(x=testfile.data['X'],
                                    y=testfile.data['Y'],
                                    z=testfile.data['Z'])
    testfile.data = testfile.data[idx >= 0]
    idx = idx[idx >= 0]

    # Create subplots
    f, axes = plt.subplots(1, 3, figsize=(12, 3.5))

    # OK plot
    gs.validation_plot(
        testfile['Cu'], OK.data['Estimate'][idx].values,
        grid=True, stat_blk='minimal', ms=2,
        title='OK Fold {}'.format(fold), figsize=(4, 4),
        ax=axes[0], vlim=(0, 1.5)
    )
    axes[0].text(
        0.19, 1.3,
        'R2: {}'.format(round(r2_score(testfile['Cu'], OK.data['Estimate'][idx].values), 2)),
        horizontalalignment='center', verticalalignment='center',
        fontsize=12, fontweight='bold'
    )

    # DeepKriging plot
    gs.validation_plot(
        testfile['Cu'], DeepKriging.data['0'][idx].values,
        grid=True, stat_blk='minimal', ms=2,
        title='DeepKriging_Fold {}'.format(fold), figsize=(4, 4),
        ax=axes[1], vlim=(0, 1.5)
    )
    axes[1].text(
        0.19, 1.3,
        'R2: {}'.format(round(r2_score(testfile['Cu'], DeepKriging.data['0'][idx].values), 2)),
        horizontalalignment='center', verticalalignment='center',
        fontsize=12, fontweight='bold'
    )

    # Cond_DK plot
    gs.validation_plot(
        testfile['Cu'], Cond_DK.data['0'][idx].values,
        grid=True, stat_blk='minimal', ms=2,
        title='DeepKriging+IDW Fold {}'.format(fold), figsize=(4, 4),
        ax=axes[2], vlim=(0, 1.5)
    )
    axes[2].text(
        0.19, 1.3,
        'R2: {}'.format(round(r2_score(testfile['Cu'], Cond_DK.data['0'][idx].values), 2)),
        horizontalalignment='center', verticalalignment='center',
        fontsize=12, fontweight='bold'
    )

    # Attempt to resolve aspect ratio warnings by resetting aspect
    for a in axes:
        a.set_aspect('auto')  # You may experiment with 'equal', 'datalim', etc.

    # Update cumulative R2 scores
    OK_R2 += r2_score(testfile['Cu'], OK.data['Estimate'][idx].values)
    DeepKriging_R2 += r2_score(testfile['Cu'], DeepKriging.data['0'][idx].values)
    Cond_DK_R2 += r2_score(testfile['Cu'], Cond_DK.data['0'][idx].values)

    # Update ensemble predictions
    OK_Ensemble += OK.data['Estimate'].values.reshape(-1, 1)
    DeepKriging_Ensemble += DeepKriging.data['0'].values.reshape(-1, 1)
    Cond_DK_Ensemble += Cond_DK.data['0'].values.reshape(-1, 1)

# Compute average performances
OK_Performance = round(OK_R2 / 5, 3)
DeepKriging_Performance = round(DeepKriging_R2 / 5, 3)
Cond_DK_Performance = round(Cond_DK_R2 / 5, 3)

# Compute average ensemble predictions
OK_Avg = OK_Ensemble / 5
DeepKriging_Avg = DeepKriging_Ensemble / 5
Cond_DK_Avg = Cond_DK_Ensemble / 5

# Write outputs
gs.write_gslib(pd.DataFrame(OK_Avg), 'OK_Average.out')
gs.write_gslib(pd.DataFrame(DeepKriging_Avg), 'DeepKriging_Average.out')
gs.write_gslib(pd.DataFrame(Cond_DK_Avg), 'Cond_DK_Average.out')


In [ ]:
print(round(OK_Performance,2))
print(round(DeepKriging_Performance,2))
print(round(Cond_DK_Performance,2))

In [ ]:
from sklearn.metrics import mean_squared_error

# Initialize RMSE values
OK_RMSE = 0
DeepKriging_RMSE = 0
Cond_DK_RMSE = 0


for fold in range(1, 6):
    testfile = gs.DataFile(flname='Validation0{}.dat'.format(fold), griddef=griddef, readfl=True)
    
    OK = gs.DataFile('OK_{}.out'.format(fold), griddef=griddef)
    OK.data['Estimate'][OK.data['Estimate'] < 0] = 0
    
    DeepKriging = gs.DataFile('DeepKriging_{}.out'.format(fold), griddef=griddef)   
    Cond_DK = gs.DataFile('./IDW/Weighted_DK_{}.out'.format(fold), griddef=griddef)
    
    idx, ingrid = griddef.get_index(x=testfile.data['X'], y=testfile.data['Y'], z=testfile.data['Z'])
    testfile.data = testfile.data[idx >= 0]
    idx = idx[idx >= 0]    
    
    
    OK_rmse = np.sqrt(mean_squared_error(testfile['Cu'], OK.data['Estimate'][idx].values))
    DeepKriging_rmse = np.sqrt(mean_squared_error(testfile['Cu'], DeepKriging.data['0'][idx].values))
    Cond_DK_rmse = np.sqrt(mean_squared_error(testfile['Cu'], Cond_DK.data['0'][idx].values))
    
    
    OK_RMSE += OK_rmse
    DeepKriging_RMSE += DeepKriging_rmse   
    Cond_DK_RMSE += Cond_DK_rmse


OK_Performance = round(OK_RMSE/5, 3)
DeepKriging_Performance = round(DeepKriging_RMSE/5, 3)    
Cond_DK_Performance = round(Cond_DK_RMSE/5, 3)  
    


print(round(OK_Performance,3))
print(round(DeepKriging_Performance,3))
print(round(Cond_DK_Performance,3))


In [ ]:
ok_results = gs.DataFile('OK_Average.out')
deepkriging_results = gs.DataFile('DeepKriging_Average.out')
Cond_DK_results = gs.DataFile('Cond_DK_Average.out')

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

# Plot the first histogram
gs.histogram_plot(
    deepkriging_results,
    var='0',
    ax=ax,
    color='blue',
    alpha=0.5,  
    stat_xy=(0.5,0.95), 
    label='Deep Kriging Results'
)

# Plot the second histogram
gs.histogram_plot(
    bhdata,
    var='Cu',
    ax=ax,
    color='orange',
    alpha=0.5,  
    label='BH Data'
)

# Customize labels, legend, and tick sizes
ax.set_xlabel("Variable Value", fontsize=14)
ax.set_ylabel("Frequency", fontsize=14)
ax.tick_params(labelsize=12)  #
plt.legend(fontsize=12)       

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

fig1 = plt.figure(figsize=(10, 4))
# Adjust the width_ratios to have three elements
gs2 = gridspec.GridSpec(2, 2, height_ratios=[1, 1])

# Define the axes using the custom gridspec layout
ax0 = fig1.add_subplot(gs2[:, 0])
ax1 = fig1.add_subplot(gs2[0, 1])
ax2 = fig1.add_subplot(gs2[1, 1])


gs.slice_plot(Cond_DK_results, var = '0', pointdata = bhdata, pointvar = 'Cu', griddef=griddef, 
              pointkws={'edgecolors': 'k', 's':5}, title = 'Cond DK Estimation - Plan View', ax = ax0,
              slice_number = 20, aspect = 1.5)

gs.slice_plot(Cond_DK_results, var = '0', pointdata = bhdata, pointvar = 'Cu', griddef=griddef, 
              pointkws={'edgecolors': 'k', 's':5}, title = 'Cond DK Estimation - E-W Section View', ax = ax1,
              slice_number = 60,  aspect = 0.8, orient = 'xz')

gs.slice_plot(Cond_DK_results, var = '0', pointdata = bhdata, pointvar = 'Cu', griddef=griddef, 
              pointkws={'edgecolors': 'k', 's':5}, title = 'Cond DK Estimation - N-S Section View', ax = ax2,
              slice_number = 40, aspect = 0.5, orient = 'yz')

# Adjust the layout spacing
plt.tight_layout(h_pad = 3, w_pad = 1.5)
plt.show()


In [ ]:
# kt3dnpar = """                 Parameters for KT3DN
#                  ********************
# START OF PARAMETERS:
# {datafl}                         -file with data
# 0  {xyzcol} {varcol}  0          -columns for DH,X,Y,Z,var,sec var
# -998.0    1.0e21                 -trimming limits
# 0                                -option: 0=grid, 1=cross, 2=jackknife
# xvk.dat                          -file with jackknife data
# 1   2   0    3    0              -columns for X,Y,Z,vr and sec var
# kt3dn_dataspacing.out            -data spacing analysis output file (see note)
# 0    15.0                        -number to search (0 for no dataspacing analysis, rec. 10 or 20) and composite length
# 0    100   0                     -debugging level: 0,3,5,10; max data for GSKV;output total weight of each data?(0=no,1=yes)
# kt3dn.dbg-nkt3dn.sum             -file for debugging output (see note)
# {outfl}                          -file for kriged output (see GSB note)
# {griddef}
# 5    5      5                    -x,y and z block discretization
# 4    48    12    1              -min, max data for kriging,upper max for ASO,ASO incr
# 0      0                        -max per octant, max per drillhole (0-> not used)
# 600.0  600.0  400.0              -maximum search radii
# 00.0   0   0.0                  -angles for search ellipsoid
# 1                                -0=SK,1=OK,2=LVM(resid),3=LVM((1-w)*m(u))),4=colo,5=exdrift,6=ICCK
# 0.180 0.6  0.8                   -mean (if 0,4,5,6), corr. (if 4 or 6), var. reduction factor (if 4)
# 0 0 0 0 0 0 0 0 0                -drift: x,y,z,xx,yy,zz,xy,xz,zy
# 0                                -0, variable; 1, estimate trend
# extdrift.out                     -gridded file with drift/mean
# 4                                -column number in gridded file
# key_out.out                      -gridded file with keyout (see note)
# 0    1                           -column (0 if no keyout) and value to keep
# -1    0                          -  nst, nugget effect
# 1   2   0.0   0.0   0.0           - it,cc,ang1,ang2,ang3
#      100    100    100            - a_hmax, a_hmin, a_vert
# """
# kt3dn = gs.Program(program = 'exe/kt3dn')

# griddef = gs.GridDef(grid_file='griddef.txt')
# griddef

# Traindat = gs.DataFile ('cudata.dat', griddef = griddef,
#                        readfl=True)

    
# kt3dn.run(parstr=kt3dnpar.format(datafl = Traindat.flname,
#                                             xyzcol = Traindat.gscol(Traindat.xyz),
#                                             varcol = Traindat.gscol('Cu'),                              
#                                             griddef = griddef,
#                                             outfl = 'IDW.out'))          
    


In [ ]:
# IDW = gs.DataFile('IDW.out')

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib import gridspec

# fig1 = plt.figure(figsize=(10, 4))
# # Adjust the width_ratios to have three elements
# gs2 = gridspec.GridSpec(2, 2, height_ratios=[1, 1])

# # Define the axes using the custom gridspec layout
# ax0 = fig1.add_subplot(gs2[:, 0])
# ax1 = fig1.add_subplot(gs2[0, 1])
# ax2 = fig1.add_subplot(gs2[1, 1])


# gs.slice_plot(IDW, var = 'Estimate', pointdata = bhdata, pointvar = 'Cu', griddef=griddef, 
#               pointkws={'edgecolors': 'k', 's':5}, title = 'Cond DK Estimation - Plan View', ax = ax0,
#               slice_number = 20, aspect = 1.5)

# gs.slice_plot(IDW, var = 'Estimate', pointdata = bhdata, pointvar = 'Cu', griddef=griddef, 
#               pointkws={'edgecolors': 'k', 's':5}, title = 'Cond DK Estimation - E-W Section View', ax = ax1,
#               slice_number = 60,  aspect = 0.8, orient = 'xz')

# gs.slice_plot(IDW, var = 'Estimate', pointdata = bhdata, pointvar = 'Cu', griddef=griddef, 
#               pointkws={'edgecolors': 'k', 's':5}, title = 'Cond DK Estimation - N-S Section View', ax = ax2,
#               slice_number = 40, aspect = 0.5, orient = 'yz')

# # Adjust the layout spacing
# plt.tight_layout(h_pad = 3, w_pad = 1.5)
# plt.show()


In [ ]:
import sys
import platform

import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import pygeostat as gs
import tqdm
import scipy
import statsmodels
import sklearn
import tensorflow as tf
import hyperopt

print("System platform:", platform.platform())
print("Python version :", sys.version)
print("NumPy version  :", np.__version__)
print("Pandas version :", pd.__version__)
print("Matplotlib version  :", mpl.__version__)
print("Seaborn version     :", sns.__version__)
print("pygeostat version   :", getattr(gs, '__version__', 'No version attribute'))
print("tqdm version        :", tqdm.__version__)
print("SciPy version       :", scipy.__version__)
print("statsmodels version :", statsmodels.__version__)
print("scikit-learn version:", sklearn.__version__)
print("TensorFlow version  :", tf.__version__)
print("hyperopt version    :", hyperopt.__version__)

# If AGD/Eikonal has a version attribute:
# from agd import Eikonal
# print("AGD/Eikonal version:", getattr(Eikonal, '__version__', 'No version attribute'))
